In [1]:
# Import the dependencies

import numpy as np
import pandas as pd
import pickle
import os
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.svm import SVC

import tensorflow
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.optimizers import Adam

In [2]:
# Function to load the predictor variables

def load_X_file():
    npy_file = "data/X.npy"
        
    npy = np.load(npy_file)    
    
    return npy

In [3]:
# Load the predictor variables

X_npy = load_X_file()
X_npy.shape

(39209, 32, 32, 3)

In [4]:
# Load the target variables

y = np.load("data/y.npy")
y.shape

(39209, 43)

In [5]:
# Create the target dataframe

y_npy = pd.DataFrame(y)
y_npy.head()

,0,1,2,3,4,5,6,7,8,9,...,33,34,35,36,37,38,39,40,41,42
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
# Split the data into training, validation, and test sets

X_train_npy, X_test_npy, y_train_npy, y_test_npy = train_test_split(X_npy, y_npy, test_size=0.1)
X_train_npy, X_val_npy, y_train_npy, y_val_npy = train_test_split(X_train_npy, y_train_npy, test_size=0.1)

In [7]:
# Create and train the Neural Network 

nn = Sequential()

nn.add(Conv2D(filters=32, strides=(2, 2), kernel_size=1, padding='same', input_shape=(32, 32, 3), activation="relu"))
nn.add(Conv2D(32, (3, 3), activation="relu"))
nn.add(MaxPooling2D(pool_size=(2, 2)))
nn.add(Dropout(0.1))

nn.add(Flatten())

nn.add(Dense(1800, activation='relu'))
nn.add(Dense(900, activation='relu'))
nn.add(Dense(512, activation='relu'))
nn.add(Dense(256, activation='relu'))
nn.add(Dense(43, activation='softmax'))
nn.summary()

nn.compile(loss=CategoricalCrossentropy(), optimizer="adam", metrics=["accuracy"])

model = nn.fit(X_train_npy, y_train_npy, validation_data=(X_val_npy, y_val_npy), epochs=100, batch_size=64)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 16, 16, 32)        128       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 14, 14, 32)        9248      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 7, 7, 32)          0         
_________________________________________________________________
dropout (Dropout)            (None, 7, 7, 32)          0         
_________________________________________________________________
flatten (Flatten)            (None, 1568)              0         
_________________________________________________________________
dense (Dense)                (None, 1800)              2824200   
_________________________________________________________________
dense_1 (Dense)              (None, 900)               1

497/497 [==============================] - 25s 50ms/step - loss: 0.0523 - accuracy: 0.9898 - val_loss: 0.1451 - val_accuracy: 0.9702
Epoch 46/100
497/497 [==============================] - 25s 49ms/step - loss: 0.0633 - accuracy: 0.9890 - val_loss: 0.3075 - val_accuracy: 0.9677
Epoch 47/100
497/497 [==============================] - 25s 49ms/step - loss: 0.0566 - accuracy: 0.9888 - val_loss: 0.1614 - val_accuracy: 0.9702
Epoch 48/100
497/497 [==============================] - 25s 50ms/step - loss: 0.0382 - accuracy: 0.9920 - val_loss: 0.1627 - val_accuracy: 0.9705
Epoch 49/100
497/497 [==============================] - 25s 50ms/step - loss: 0.0342 - accuracy: 0.9927 - val_loss: 0.2552 - val_accuracy: 0.9714
Epoch 50/100
497/497 [==============================] - 25s 50ms/step - loss: 0.0495 - accuracy: 0.9906 - val_loss: 0.2244 - val_accuracy: 0.9603
Epoch 51/100
497/497 [==============================] - 25s 50ms/step - loss: 0.0666 - accuracy: 0.9881 - val_loss: 0.1791 - val_accuracy

In [8]:
# Create the predicted target values dataframe

y_pred = nn.predict(X_test_npy)

In [9]:
# Convert the predicted values from decimal values between 0 and 1 to binary values of 0 or 1

y_pred_df = pd.DataFrame((y_pred+0.5).astype(int)).reset_index(drop=True)

# Create the test target variables dataframe
y_test_df = pd.DataFrame(y_test_npy)

In [10]:
# Show the number of columns in test and predicted target variables

y_pred_df.shape[1], y_test_df.shape[1]

(43, 43)

In [11]:
# Compare the predicted vs test target values

y_pred_df_tr = y_pred_df.transpose()
y_test_df_tr = y_test_df.transpose()

print(y_pred_df_tr.shape)
print(y_test_df_tr.shape)

mismatches = []

for n in range(y_pred_df.shape[0]):
    row_pred = y_pred_df.iloc[n]
    row_test = y_test_df.iloc[n]
    
    for m in range(len(row_pred)):
        if row_pred[m] != row_test[m]:
            mismatches.append(n)
            break
            
print(f"Accuracy: {(y_pred_df.shape[0] - len(mismatches))/y_pred_df.shape[0]}")


(43, 3921)
(43, 3921)
Accuracy: 0.9734761540423361
